In [1]:
from sandbox import *

In [2]:
random_state = 42

# Data loading

In [3]:
# Since we do cross validation split ourselves we join TRAINING/VALIDATION together

df_train = pd.concat([pd.read_csv('exercise_data/human_dna_train_split.csv', header=0, names =['seq', 'label']),
                       pd.read_csv('exercise_data/human_dna_validation_split.csv', header=0, names =['seq', 'label'])])
y_train = df_train.label
x_train = df_train.seq

In [4]:
# TRAINING DATASET
# y_train=y_train[:10000]
# x_train=x_train[:10000]
x_prep, y_prep = prep_data_cnn(x_train, y_train)

print(x_prep.shape)
print(y_prep.shape)
print((np.asarray(x_prep)).shape)

(10000, 398, 4)
(10000,)
(10000, 398, 4)


In [5]:
# TESTING DATASETS
df_test = pd.read_csv('exercise_data/human_dna_test_split.csv', header=0, names =['seq', 'label'])
df_test_hidden = pd.read_csv('exercise_data/human_dna_test_hidden_split.csv', header=0, names =['seq'])
y_test = df_test.label
x_test = df_test.seq
x_testh = df_test_hidden.seq

In [6]:
x_prep_test, y_prep_test = prep_data_cnn(x_test, y_test)

print(x_prep_test.shape)
print(y_prep_test.shape)
print((np.asarray(x_prep_test)).shape)

(33333, 398, 4)
(33333,)
(33333, 398, 4)


/home/flurin/machine-learning-for-health-care-project4/sandbox.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y==-1] = 0


In [7]:
x_prep_testh = prep_data_cnn(x_testh, use_y=False)

print(x_prep_testh.shape)

(30021, 398, 4)


# Analysis

In [8]:
# Settings is a list of lists containing our evaluation settings.
# We optimize modeltype, batch_size and learning_rate.


def get_settings():
    settings = []
    for m in ['cnn', 'cnn2', 'rnn']:
        for l in [0.01, 0.001]:
            for b in [64]:
                settings.append([m, l, b])
    return settings

In [9]:
# We generate a learning setting list (notice this one is considerably shorter than our evalutation on the elegans set. But with leonhard down we are only capable of optimizing on our local machine.)

settings = get_settings()
print(settings)

[['cnn', 0.01, 64], ['cnn', 0.001, 64], ['cnn2', 0.01, 64], ['cnn2', 0.001, 64], ['rnn', 0.01, 64], ['rnn', 0.001, 64]]


In [10]:
cv = cv, models = cross_validation(x_prep, y_prep, settings, 
                                   x_test=x_prep_test, y_test=y_prep_test, 
                                   x_testh=x_prep_testh, 
                                   k=5, epochs=1, test_hidden=True)  # Here we only do 3 epochs (more data, no clusters available) 

 85/125 [===================>..........] - ETA: 1s - loss: 1.2309 - accuracy: 0.9917

KeyboardInterrupt: 

# Visualization